# Exercice 3 (niveau avancé) - Classifier des apparats

Le fichier `exercice_3.xml` consiste en une table d'alignement d'un fragment du *De Regimine Principum* de Gilles de Rome. 

L'exercice guidé consistera en la classification de chaque lieu variant en utilisant les informations grammaticales et lexicales issues de l'annotation linguistique réalisée en amont (lemmes, parties du discours). 

Le but est d'ajouter à chaque élément `app` un attribut `@type` afin de permettre de filtrer et d'identifier ensuite les variantes significatives.

In [107]:
import lxml.etree as etree
import copy

tei_uri = "http://www.tei-c.org/ns/1.0"
namespaces_dict = {'tei': tei_uri}

fichier = '../exercice_3.xml'
document_as_xml = etree.parse(fichier)

## Étape 1: récupération des informations
Ici, on veut récupérer l'ensemble des informations utiles dans notre élément `tei:app`: chaînes de caractères, lemmes, parties du discours. On travaille donc apparat par apparat.

On commence par créer une requête qui nous permet de récupérer tous nos apparats:

In [94]:
all_apps = document_as_xml.xpath("//tei:app", namespaces=namespaces_dict)
len(all_apps)

655

Nous allons devoir comparer ce qui est comparable: chaque type d'information entre elle (formes, parties du discours, lemmes). Il faut donc récupérer ces informations, et on va le faire sous la forme de listes, ce qui sera très pratique pour la suite. Par chance, comme on le sait, la méthode `xpath()` renvoie précisément des listes!

**NB:** l'encodage de ces apparats n'est pas complètement conforme à la TEI. Les attributs permettant l'analyse linguistique `@pos` et `@lemma` sont indiqués au niveau du `tei:rdg`.

In [95]:
first_set_of_lemmas = all_apps[0].xpath("descendant::tei:rdg/@lemma", namespaces=namespaces_dict)
print(first_set_of_lemmas)

['capítulo', 'capítulo', 'capítulo']


## Étape 2: algorithme de classification

Ici, on veut produire une méthode de classification qui utilise les informations extraites auparavant. On définit ainsi un certain nombre de variantes (voir Camps, J.-B., Ing, L., & Spadini, E. (2019). Collating Medieval Vernacular Texts : Aligning Witnesses, Classifying Variants. DH2019 Digital Humanities Conference. DH2019: Complexities, Utrecht. https://dh-abstracts.library.cmu.edu/works/10074):
- lexicale (les lemmes diffèrent)
- graphique (seules les chaînes de caractères diffèrent)
- grammaticale (les parties du discours diffèrent)
- omission (un témoin ou plus omet le texte)

Ces classes peuvent être combinées: on peut avoir une variante qui contient à la fois une omission et une variante lexicale.

Commençons donc sur notre algorithme, par les informations que nous avons en entrée. Nous allons comparer chaque type d'information une à une: les chaînes de caractères, les lemmes, les parties du discours. Puis nous comparerons le résultat de cette première phase de comparaison afin de produire notre classification.

Nous voulons idéalement des listes (on s'occupera plus tard de la manière de les produire) pour chaque lieu variant.

In [96]:
liste_lemmes = ['a', 'a', 'a', 'b']
liste_pos = ['b', 'b', 'b', 'b']
liste_formes = ['a', 'e', 'e', 'i']

Au sein du lieu variant `tei:app`, chaque rdg viendra nourrir ces liste des informations correspondantes. Le but d'abord est d'identifier si tous les témoins partagent la même information ou s'il y a une variation d'un ou plusieurs témoin. 

Il nous faut trouver une méthode pour identifier cette variation. Nous voulons nous assurer que tous les éléments de la liste sont identiques, il faut donc parvenir à les comparer entre eux. Plusieurs méthodes sont possibles, j'en montrerai deux ici

#### En utilisant `all()` et une compréhension de liste

Une compréhension de liste revient à aplatir une boucle pour produire une liste:

In [97]:
liste_de_nombres = []
for number in range(5):
    liste_de_nombres.append(number + 10)
print(liste_de_nombres)

[10, 11, 12, 13, 14]


Est équivalent à:

In [98]:
liste_de_nombres = [number + 10 for number in range(5)]
print(liste_de_nombres)

[10, 11, 12, 13, 14]


La fonction built-in `all()` permet de vérifier que tous les éléments d'un itérable sont vrais. Elle renvoie un booléen (`True` ou `False`).

Exemple:

In [99]:
list_of_elements_a = [1, 2, 3, 4, 5]
list_of_elements_b = [1, 2, 3, 4, 5.5]
print(all([type(element) == int for element in list_of_elements_a]))
print(all([type(element) == int for element in list_of_elements_b]))

True
False


Pour vérifier si la liste est homogène, on pourrait produire une double boucle afin de comparer chacun de ses items avec tous les autres, mais cela serait un peu complexe. Une autre solution est de vérifier que chaque item de la liste est identique au premier. Si tel est le cas, ils seront tous identiques entre eux !

In [100]:
premier_lemme = liste_lemmes[0]
premier_pos = liste_pos[0]
premiere_forme = liste_formes[0]
egalite_lemmes = all(lemme == premier_lemme for lemme in liste_lemmes[1:])
egalite_pos = all(pos == premier_pos for pos in liste_pos[1:])
egalite_formes = all(forme == premiere_forme for forme in liste_formes[1:])
print(egalite_lemmes)
print(egalite_pos)
print(egalite_formes)

False
True
False


Ici, nous avons une inégalité des lemmes, une inégalité des lemmes et une inégalité des formes. En combinant ces informations, nous avons de quoi produire notre classification !

#### Avec des ensembles
Un ensemble (`set`) est un objet itérable non ordonné dont les composants sont uniques. Passer d'une liste à un ensemble (qui est une opération triviale pour l'utilisateur.ice) permet de tester l'homogénéité de cette liste, et d'identifier le nombre d'éléments différents qui la composent:

In [101]:
liste_de_fruits = ["pomme", "poire", "banane", "banane", "pomme"]
set(liste_de_fruits)

{'banane', 'poire', 'pomme'}

On peut donc utiliser cette méthode pour vérifier combien d'éléments différents sont dans nos listes d'annotations: 

In [102]:
print(set(liste_lemmes))
print(set(liste_pos))
print(set(liste_formes))

{'b', 'a'}
{'b'}
{'e', 'i', 'a'}


On valide bien ici que la liste de parties du discours contient deux annotations différentes, et qu'il y a une variation de ce point de vue.

### Classifier les variantes
Nous savons maintenant si la variation apparaît au niveau de la forme, de la partie du discours ou du lemme. En combinant ces informations, nous allons pouvoir réaliser notre classification:
1) Si seule la forme diverge, il s'agit d'une variante graphique
2) Si seule la partie du discours diverge, il s'agit d'une variante grammaticale
3) Si le lemme change, il s'agit d'une variante lexicale

On va pouvoir ici utiliser des formes conditionelles pour exprimer ces différentes options. Nous allons utiliser la structure conditionnelle `if... elif` qui permet de tester de multiples conditions à la suite. Ici, on simplifie la vérification de la valeur d'une variable: `not egalite_formes` est équivalent (et préférable) à `egalite_formes == False` ou `egalite_formes is False`.

In [103]:
variante = ""
if not egalite_formes and egalite_lemmes and egalite_pos:
    variante = "graphique"
elif egalite_lemmes and not egalite_pos:
    variante = "grammaticale"
elif not egalite_lemmes:
    variante = "lexicale"
print(variante)

lexicale


Nous avons maintenant l'ensemble des étapes d'identification de la variante: nous pouvons en faire une fonction qui prendra les listes en entrée et qui produira en sortie la classe.

Nous allons simplement devoir complexifier un peu la fonction en ajoutant une manière de gérer les omissions: 

In [104]:
first_set_of_lemmas = all_apps[20].xpath("descendant::tei:rdg/@lemma", namespaces=namespaces_dict)
print(first_set_of_lemmas)

['le', '']


Ceci sera fait en identifiant les listes contenant des éléments vides, qui mèneront à une classe supplémentaire, `variante.` En même temps, on supprimera ces éléments vides qui viendraient fausser la classification.

In [105]:
# Les éléments qui apparaissent en vert dans la première ligne correspondent à une forme de documentation de la fonction
def classer_variantes(formes:list, parties_discours:list, lemmes:list)-> str:
    variante = ""
    if "" in parties_discours:
        variante = "#omission "
        # On réécrit les listes à l'aide de comprehensions de listes qui incluent une condition
        parties_discours = [item for item in parties_discours if item != ""]
        lemmes = [item for item in lemmes if item != ""]
    egalite_lemmes = all(lemme == lemmes[0] for lemme in lemmes[1:])
    egalite_pos = all(pos == parties_discours[0] for pos in parties_discours[1:])
    egalite_formes = all(forme == formes[0] for forme in formes[1:])
    if not egalite_formes and egalite_lemmes and egalite_pos:
        variante += "#graphique"
    elif egalite_lemmes and not egalite_pos:
        variante += "#grammaticale"
    elif not egalite_lemmes:
        variante += "#lexicale"
    # On s'occupe du cas où il n'y aurait qu'une omission (deux rdg seulement)
    if variante == "#omission ": variante = "#omission"
    return variante

Nous pouvons maintenant tout réunir dans une boucle !

## Étape 3: réalisation de la chaîne de traitement

Chaque entrée d'apparat doit être typée à l'aide d'un attribut `@ana` (attention, cet attribut doit être un pointeur). 

In [106]:
for app in all_apps:
    formes = app.xpath("descendant::tei:rdg/tei:w/text()", namespaces=namespaces_dict)
    parties_discours = app.xpath("descendant::tei:rdg/@pos", namespaces=namespaces_dict)
    lemmes = app.xpath("descendant::tei:rdg/@lemma", namespaces=namespaces_dict)
    variante = classer_variantes(formes, parties_discours, lemmes)
    app.set('ana', variante)

with open('../edition_apparat_type.xml', 'w') as output_file:
    output_file.write(etree.tostring(document_as_xml, pretty_print=True).decode())
    